Importing libraries

In [1]:
import torch
import numpy as np
from dataset import Dataset
from arcface import iresnet50
from prediction import predict
from torchvision import transforms
from evaluation import evaluate

Model loading

In [2]:
model = iresnet50()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
state_dict = torch.load('/home/elobaida/insightface/recognition/arcface_torch/work_dirs/custom/model.pt', map_location=device)
model.load_state_dict(state_dict, strict=False)
model.eval()
model = model.cuda()

Data transforms and dataloader

In [13]:
transformations = transforms.Compose([
transforms.Resize((112, 112)),
transforms.ToTensor(),           
transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),    # Normalize the pixel values
    ])

dataloader = torch.utils.data.DataLoader(
    dataset=Dataset(
        dir='/home/elobaida/prediction/bfw-cropped-aligned/white_males',
        pairs_path='/home/elobaida/prediction/bfw_pairs/pairs_white_males_bfw.txt',
        transform=transformations
    ),
    batch_size=200,
    num_workers=4,
    shuffle=False,
    pin_memory=False
)

Prediction

In [14]:
distances, labels = \
predict(model=model, \
        dataloader=dataloader)

  0%|          | 0/447 [00:00<?, ?it/s]

Testing on IJB! ...


100%|██████████| 447/447 [04:43<00:00,  1.57it/s]


Evaluation

In [17]:
# with open('./predictions/dist_labels_white_males_bfw.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
#             distances, labels = pickle.load(f)

In [18]:
distances = np.concatenate(distances).ravel()
labels = np.concatenate(labels).ravel()

In [19]:
_, _, _, _, _, eer = evaluate(
                        labels=labels,
                        distances=distances,
                        ts=0,
                        fmr_p=0.001
                    )

------------------------------------------------Single fold---------------------------------------

Accuracy: 0.9993	Precision 0.9762	Recall 0.8227	ROC Area Under Curve: 0.9861	
fpr at tpr 0.95: 0.08199775533108866,  tpr at fpr 0.001: 0.8729096989966555, tpr at fpr 0.0001: 0.8260869565217391
At FNR = FPR: FNR = 0.056856187290969896, FPR = 0.05768799102132435
